In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import keras_preprocessing

train_dir = '/content/drive/MyDrive/Dataset Fix'


train_datagen = keras_preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    subset='training')

# Create a validation generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    subset='validation')

Found 20 images belonging to 4 classes.
Found 4 images belonging to 4 classes.


In [4]:
pre_trained_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [5]:
pre_trained_model.trainable = False

model = tf.keras.Sequential([
    pre_trained_model,  
    tf.keras.layers.Conv2D(32, 3, activation='relu'),  
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.GlobalAveragePooling2D(),  
    tf.keras.layers.Dense(4, activation='softmax')  
])


In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 132       
Total params: 2,626,788
Trainable params: 368,804
Non-trainable params: 2,257,984
_________________________________________________________________


In [8]:
history = model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
1/1 [==============================] - 1s 947ms/step - loss: 0.0779 - accuracy: 1.0000 - val_loss: 0.0754 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 1s 928ms/step - loss: 0.0756 - accuracy: 1.0000 - val_loss: 0.0628 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 1s 901ms/step - loss: 0.0613 - accuracy: 1.0000 - val_loss: 0.0541 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 1s 865ms/step - loss: 0.0618 - accuracy: 1.0000 - val_loss: 0.0474 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 1s 952ms/step - loss: 0.0543 - accuracy: 1.0000 - val_loss: 0.0419 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 1s 959ms/step - loss: 0.0449 - accuracy: 1.0000 - val_loss: 0.0369 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 1s 982ms/step - loss: 0.0463 - accuracy: 1.0000 - val_loss: 0.0323 - val_accuracy: 1.0000
Epoch 8/20
1/1 [====

In [9]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 4.9MB/s 


In [11]:
import numpy as np
from keras.preprocessing import image
from extracted_face import extract_face 

path = '/content/drive/MyDrive/Face Dataset/Validation/M0101080/S__87277580.jpg'
img = extract_face(path)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
predictions = model.predict(images/255.0)
score = tf.nn.softmax(predictions[0])
print(predictions[0])

[9.9999416e-01 1.2813607e-06 1.6452015e-09 4.5883921e-06]


In [ ]:
model.save('/content/my_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
